# Práctica Final Aprendizaje Automático

Grupo 5: Jorge Ortega y Daniela Vidal

## Índice

1. [Presentación del dataset](#Presentación-del-dataset)
2. [Preprocesamiento](#Preprocesamiento)
3. [Dataset final](#Dataset-final)
4. [Separación en entrenamiento, validación y test](#Separación-en-entrenamiento,-validación-y-test)
5. [Red logística](#Red-logística)
6. [Red neuronal](#Red-neuronal)
7. [Árboles de decisión](#Árboles-de-decisión)
8. [Comparación de resultados](#Comparación-de-resultados)
9. [Conclusiones](#Conclusiones)
10. [Bibliografía](#Bibliografía) 

## 1. Presentación del dataset <a class="anchor" id="Presentación-del-dataset"></a>

Esta práctica consiste en la aplicación de los conocimientos adquiridos en la asignatura de Aprendizaje Automático a un dataset sobre reconocimiento de gestos con datos de la actividad muscular. Lo que se pretende es realizar un clasificador que sea capaz de predecir qué gesto se está realizando a partir de los datos de la actividad muscular.

El dataset que hemos elegido para realizar la práctica es el de [Classify gestures by reading muscle activity](https://www.kaggle.com/datasets/kyr7plus/emg-4). Se trata de un dataset de Kaggle que contiene datos de la actividad muscular de 4 gestos diferentes: 
- Piedra (puño)

- Papel (extensión de la mano)

- Tijeras (dedos índice y corazón extendidos)

- Okay (dedo índice tocando el pulgar y el resto de dedos extendidos). 

Los datos se han recogido con señales EMG (electromiografía). Las señales EMG son señales eléctricas que se producen en los músculos cuando estos se activan y se pueden utilizar para detectar y clasificar gestos. Los datos se han recogido con una banda colocada en el brazo con 8 sensores en la superficie de la piel, cada uno de ellos recoge la actividad eléctrica producida por los músculos debajo de la piel. 
Cada línea del dataset consiste en 40ms en las que se hacen 8 medidas consecutivas de los 8 sensores, es decir, 64 columnas en total de esta manera:

                                    [8 sensores][8 sensores][8 sensores][8 sensores][8 sensores][8 sensores][8 sensores][8 sensores][Gesto]

En total hay 120s de cada gesto.

La columna 'Gesto' clasifica piedra - 0, tijeras - 1, papel - 2, okay - 3.

Para asegurarse de la calidad de los datos, se realizaron 6 repeticiones de cada gesto en la que se empezaba y se paraba de medir con el gesto sostenido.

Se trata de 4 datasets de alrededor de 2900 filas cada uno con la actividad de cada gesto. 

## 2. Preprocesamiento <a class="anchor" id="Preprocesamiento"></a>

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import tensorflow as tf

### Constantes globales

In [2]:
RANDOM_STATE = 42

### Lectura del dataset

In [3]:
X0 = pd.read_csv('0.csv')
X1 = pd.read_csv('1.csv')
X2 = pd.read_csv('2.csv')
X3 = pd.read_csv('3.csv')

Podemos ver lo que contienen los datasets en las siguientes tablas:

In [4]:
X0.head()

,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,...,-28.0,61.0,4.0.3,8.0.1,5.0.1,4.0.4,-7.0.1,-59.0,16.0,0
0,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,...,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
1,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,...,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
2,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,...,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
3,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,...,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0
4,15.0,-5.0,-5.0,-15.0,12.0,-22.0,-38.0,36.0,9.0,6.0,...,-26.0,5.0,6.0,6.0,11.0,5.0,30.0,-48.0,25.0,0


In [5]:
X1.head()

,-7.0,-1.0,-1.0.1,0.0,-10.0,-10.0.1,-1.0.2,1.0,-5.0,-5.0.1,...,6.0.1,-4.0.6,-3.0.5,-5.0.5,-3.0.6,15.0.1,11.0,-4.0.7,-5.0.6,1
0,-6.0,-2.0,-5.0,-2.0,27.0,42.0,3.0,5.0,11.0,1.0,...,2.0,-17.0,-5.0,-7.0,-2.0,15.0,12.0,0.0,-7.0,1
1,5.0,0.0,-1.0,-2.0,2.0,-9.0,1.0,5.0,1.0,-3.0,...,-11.0,-23.0,1.0,-1.0,-1.0,-23.0,-25.0,2.0,-1.0,1
2,31.0,4.0,2.0,-2.0,38.0,14.0,2.0,7.0,-2.0,4.0,...,-4.0,13.0,2.0,-1.0,-3.0,-7.0,0.0,-3.0,-2.0,1
3,-4.0,-4.0,3.0,3.0,-25.0,-46.0,-1.0,3.0,-7.0,-6.0,...,15.0,4.0,-4.0,-1.0,-1.0,7.0,26.0,3.0,-7.0,1
4,-8.0,-3.0,-1.0,0.0,14.0,5.0,-4.0,-6.0,2.0,0.0,...,-18.0,53.0,8.0,-1.0,-1.0,-2.0,-5.0,-2.0,9.0,1


In [6]:
X2.head()

,4.0,19.0,-9.0,-7.0,-3.0,-36.0,-6.0,-23.0,3.0,-21.0,...,9.0,-14.0.1,-2.0.5,-3.0.3,-4.0,-21.0.1,7.0,-8.0.1,-12.0.2,2
0,-1.0,12.0,20.0,7.0,20.0,-73.0,-4.0,-2.0,4.0,5.0,...,-3.0,-5.0,-2.0,0.0,-4.0,-7.0,5.0,6.0,9.0,2
1,4.0,5.0,-8.0,-2.0,10.0,-10.0,-10.0,16.0,-3.0,-18.0,...,2.0,0.0,2.0,3.0,-8.0,19.0,20.0,0.0,-8.0,2
2,-3.0,-3.0,5.0,11.0,25.0,-20.0,-2.0,14.0,9.0,32.0,...,15.0,-2.0,16.0,6.0,9.0,1.0,31.0,16.0,4.0,2
3,-5.0,-9.0,-2.0,-5.0,-46.0,-34.0,-9.0,-4.0,0.0,8.0,...,-5.0,-11.0,-6.0,7.0,6.0,-10.0,-24.0,-6.0,0.0,2
4,3.0,-3.0,-6.0,-5.0,-3.0,-4.0,-5.0,-4.0,-5.0,8.0,...,-18.0,-14.0,8.0,1.0,-2.0,-4.0,-34.0,-3.0,6.0,2


In [7]:
X3.head()

,-22.0,-9.0,-6.0,-1.0,21.0,26.0,-2.0,5.0,5.0.1,0.0,...,-10.0,-15.0.1,-6.0.3,-3.0.4,3.0.1,20.0,25.0,1.0.5,1.0.6,3
0,-7.0,0.0,1.0,0.0,-13.0,-18.0,0.0,-9.0,-6.0,-2.0,...,-1.0,11.0,-4.0,-5.0,-4.0,3.0,-8.0,-7.0,-3.0,3
1,-6.0,-6.0,-6.0,-8.0,25.0,43.0,-4.0,7.0,6.0,1.0,...,3.0,2.0,1.0,3.0,-1.0,1.0,9.0,4.0,4.0,3
2,2.0,1.0,1.0,1.0,0.0,-23.0,2.0,-6.0,-1.0,-2.0,...,16.0,-3.0,-1.0,-1.0,-3.0,0.0,-3.0,4.0,3.0,3
3,0.0,3.0,4.0,5.0,1.0,25.0,-9.0,0.0,0.0,1.0,...,1.0,6.0,2.0,1.0,-3.0,-20.0,-42.0,-4.0,2.0,3
4,-11.0,-2.0,-5.0,1.0,-7.0,-11.0,-4.0,-23.0,-9.0,0.0,...,-4.0,9.0,1.0,2.0,8.0,21.0,41.0,15.0,9.0,3


Como habíamos comentado, cada dataset tiene 2900 filas, 8 columnas de sensores y una columna de gesto. En este caso, los gestos están codificados como 0, 1, 2 y 3 para piedra, tijeras, papel y okay respectivamente. En el resto de columnas podemos ver datos numéricos que representan la actividad muscular en cada sensor.

In [8]:
print(X0.shape, X1.shape, X2.shape, X3.shape)

(2909, 65) (2902, 65) (2942, 65) (2921, 65)


Como podemos hay más o menos el mismo número de gestos de cada tipo y el mismo número de medidas. Igualmente nos encargaremos del desbalanceo de clases para obtener mejores resultados.

A continuación vamos a comprobar si para cada archivo está correctamente clasificada su clase:

In [9]:
X0.iloc[:,-1].value_counts(), X1.iloc[:,-1].value_counts(), X2.iloc[:,-1].value_counts(), X3.iloc[:,-1].value_counts()

(0    2909
 Name: 0, dtype: int64,
 1    2902
 Name: 1, dtype: int64,
 2    2942
 Name: 2, dtype: int64,
 3    2921
 Name: 3, dtype: int64)

Comprobaremos también que no hay valores nulos en los datasets:

In [10]:
X0.isna().sum().sum(), X1.isna().sum().sum(), X2.isna().sum().sum(), X3.isna().sum().sum()

(0, 0, 0, 0)

Ahora que hemos comprobado que los datos están bien clasificados y no hay valores nulos, vamos a unir los 4 datasets en uno solo para poder trabajar con todos los datos a la vez:

In [11]:
allFiles=['0.csv','1.csv','2.csv','3.csv']

list = []
for file in allFiles:
    read = pd.read_csv(file, header = None)
    list.append(read)
data = pd.concat(list)


In [12]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,26.0,4.0,5.0,8.0,-1.0,-13.0,-109.0,-66.0,-9.0,2.0,...,-28.0,61.0,4.0,8.0,5.0,4.0,-7.0,-59.0,16.0,0
1,-47.0,-6.0,-5.0,-7.0,13.0,-1.0,35.0,-10.0,10.0,-4.0,...,-25.0,47.0,6.0,6.0,5.0,13.0,21.0,111.0,15.0,0
2,-19.0,-8.0,-8.0,-8.0,-21.0,-6.0,-79.0,12.0,0.0,5.0,...,-83.0,7.0,7.0,1.0,-8.0,7.0,21.0,114.0,48.0,0
3,2.0,3.0,0.0,2.0,0.0,22.0,106.0,-14.0,-16.0,-2.0,...,-38.0,-11.0,4.0,7.0,11.0,33.0,39.0,119.0,43.0,0
4,6.0,0.0,0.0,-2.0,-14.0,10.0,-51.0,5.0,7.0,0.0,...,38.0,-35.0,-8.0,2.0,6.0,-13.0,-24.0,-112.0,-69.0,0


Por otro lado podemos ver como los rangos de los datos son relativamente diferentes, por lo que tendremos que normalizarlos para que no haya problemas a la hora de entrenar los modelos.

In [13]:
def min_max_scale(X):
    x = X.iloc[:, :-1].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df_scaled = pd.DataFrame(x_scaled, columns=X.columns[:-1]) #La columna de la clase no se escala
    df_scaled[X.columns[-1]] = X[X.columns[-1]].values
    return df_scaled

In [14]:
X_scal = min_max_scale(data)

## 3. Dataset final <a class="anchor" id="Dataset-final"></a>

El dataset final que vamos a utilizar para entrenar los modelos es el siguiente:

In [15]:
X_scal.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0.625551,0.556701,0.567164,0.638462,0.563380,0.437751,0.074510,0.244094,0.426160,0.555556,...,0.413223,0.728395,0.566524,0.720000,0.632,0.502347,0.474510,0.270588,0.557769,0
1,0.303965,0.505155,0.417910,0.523077,0.629108,0.485944,0.639216,0.464567,0.506329,0.529915,...,0.425620,0.670782,0.575107,0.693333,0.632,0.544601,0.584314,0.937255,0.553785,0
2,0.427313,0.494845,0.373134,0.515385,0.469484,0.465863,0.192157,0.551181,0.464135,0.568376,...,0.185950,0.506173,0.579399,0.626667,0.528,0.516432,0.584314,0.949020,0.685259,0
3,0.519824,0.551546,0.492537,0.592308,0.568075,0.578313,0.917647,0.448819,0.396624,0.538462,...,0.371901,0.432099,0.566524,0.706667,0.680,0.638498,0.654902,0.968627,0.665339,0
4,0.537445,0.536082,0.492537,0.561538,0.502347,0.530120,0.301961,0.523622,0.493671,0.547009,...,0.685950,0.333333,0.515021,0.640000,0.640,0.422535,0.407843,0.062745,0.219124,0


In [16]:
X_scal.shape

(11678, 65)

Es decir 11678 filas y 65 columnas.

In [18]:
X_scal.iloc[:,-1].value_counts()

2    2943
3    2922
0    2910
1    2903
Name: 64, dtype: int64

## 4. Separación en entrenamiento, validación y test <a class="anchor" id="Separación-en-entrenamiento,-validación-y-test"></a>

Dividiremos en 60-20-20 para entrenamiento, test y validación. Aunque las clases están balanceadas, utilizaremos la función de sklearn para asegurarnos de que se mantienen las proporciones de cada clase en cada conjunto.

In [21]:
X_train, X, y_train, y = train_test_split(X_scal.iloc[:, :-1], X_scal.iloc[:, -1],test_size=0.4, random_state=RANDOM_STATE, stratify=X_scal.iloc[:, -1])
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5, random_state=RANDOM_STATE, stratify=y)

In [22]:
X_train.shape, X_val.shape, X_test.shape

((7006, 64), (2336, 64), (2336, 64))

## 5. Regresión logística <a class="anchor" id="Reg-logística"></a>

Para la regresión logística usaremos solo 2 de los 4 gestos para poder comparar los resultados con los de la red neuronal y los árboles de decisión. En este caso usaremos piedra y papel.

In [32]:
allFiles=['0.csv','1.csv']

list = []
for file in allFiles:
    read = pd.read_csv(file, header = None)
    list.append(read)
data_reg = pd.concat(list)

In [33]:
X_reg_scal = min_max_scale(data_reg)

In [38]:
X_r_train, X_r, y_r_train, y_r = train_test_split(X_reg_scal.iloc[:, :-1], X_reg_scal.iloc[:, -1],test_size=0.4, random_state=RANDOM_STATE, stratify=X_reg_scal.iloc[:, -1])
X_r_val, X_r_test, y_r_val, y_r_test = train_test_split(X_r, y_r, test_size=0.5, random_state=RANDOM_STATE, stratify=y_r)

In [39]:
def func_sigmoide(x, w, b):
    z = np.dot(x,w)+b
    return 1/(1+np.exp(-z))

In [40]:
def predict(X, Y, w, b):
   # XX = map_feature(X[:, 0], X[:, 1])
    y_hat= func_sigmoide(X, w, b)
    y_hat = np.where(y_hat >= 0.5, 1, 0)
    return y_hat

In [41]:
def func_coste(X, Y, w, b, L):
    m = len(Y)
    f = func_sigmoide(X, w, b)
    coste = (-1/m)*np.sum(Y*np.log(f)+(1-Y)*np.log(1-f))
    coste += (L/(2*m))*np.sum(w**2)
    return coste

In [42]:
def funcion_grad(X, y, w, b, L):
    m = len(y)
    gw = np.zeros(w.shape)
    gb = 0
    y_hat = func_sigmoide(X, w, b)
    dif = y_hat-y
    
    gb = np.sum(dif)/m                                    
    gw = (1/m) * np.dot(dif,X) + L*w/m
    
    return gw, gb

In [43]:
def funcion_grad_desc(X, y, w, b, alpha, num_iter, L):   
    for i in range(num_iter):
        gw, gb = funcion_grad(X, y, w, b, L)
        w = w - alpha*gw
        b = b - alpha*gb
    return w, b

In [50]:
def regresion_logistica(X, y, alpha, num_iter, L):
    w = np.zeros(X.shape[1])
    b = 0
    w, b = funcion_grad_desc(X, y, w, b, alpha, num_iter, L)
    return w, b

In [51]:
Ls = [0.1, 1, 2, 4]

for l in Ls:
    w, b = funcion_grad_desc(X_r_train, y_r_train, w, b, 0.001, 100000, l)
    print("Lambda: ", l)
    y_hat_train = predict(X_r_train, y_r_train, w, b)
    print("Accuracy: ", np.mean(y_hat_train == y_r_train)*100, "%")
    y_hat = predict(X_r_test, y_r_test, w, b)
    print("Accuracy: ", np.mean(y_hat == y_r_test)*100, "%")

Lambda:  0.1
Accuracy:  59.994264410668194 %
Accuracy:  57.007738607050726 %
Lambda:  1
Accuracy:  60.02294235732721 %
Accuracy:  58.21152192605331 %
Lambda:  2
Accuracy:  60.48178950387152 %
Accuracy:  58.38349097162511 %
Lambda:  4
Accuracy:  60.768568970461715 %
Accuracy:  58.89939810834049 %


In [52]:
Ls = [6, 8, 10]

for l in Ls:
    w, b = funcion_grad_desc(X_r_train, y_r_train, w, b, 0.001, 100000, l)
    print("Lambda: ", l)
    y_hat_train = predict(X_r_train, y_r_train, w, b)
    print("Accuracy: ", np.mean(y_hat_train == y_r_train)*100, "%")
    y_hat = predict(X_r_test, y_r_test, w, b)
    print("Accuracy: ", np.mean(y_hat == y_r_test)*100, "%")

Lambda:  6
Accuracy:  60.940636650415826 %
Accuracy:  58.89939810834049 %
Lambda:  8
Accuracy:  61.28477201032406 %
Accuracy:  58.89939810834049 %
Lambda:  10
Accuracy:  61.37080585030111 %
Accuracy:  58.8134135855546 %
